In [1]:
from get_your_ans import *

2025-03-09 09:15:05,158	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.6.5 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [7]:
from vllm import LLM, SamplingParams

# Load an open-source LLM with vLLM (change model if needed)
qa_model = LLM(model="mistralai/Mistral-7B-Instruct-v0.2", dtype="float16",tensor_parallel_size= 2) 

INFO 03-09 09:17:48 __init__.py:207] Automatically detected platform cuda.
WARNING 03-09 09:17:48 config.py:2448] Casting torch.bfloat16 to torch.float16.
INFO 03-09 09:18:00 config.py:549] This model supports multiple tasks: {'score', 'reward', 'generate', 'embed', 'classify'}. Defaulting to 'generate'.
INFO 03-09 09:18:00 config.py:1382] Defaulting to use mp for distributed inference
INFO 03-09 09:18:00 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='mistralai/Mistral-7B-Instruct-v0.2', speculative_config=None, tokenizer='mistralai/Mistral-7B-Instruct-v0.2', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=Deco

/root/Anaconda/lib/python3.10/site-packages/vllm/transformers_utils/tokenizer_group/tokenizer_group.py:25: FutureWarning: It is strongly recommended to run mistral models with `--tokenizer-mode "mistral"` to ensure correct encoding and decoding.
  self.tokenizer = get_tokenizer(self.tokenizer_id, **tokenizer_config)


WARNING 03-09 09:18:01 utils.py:2128] CUDA was previously initialized. We must use the `spawn` multiprocessing start method. Setting VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/getting_started/troubleshooting.html#python-multiprocessing for more information.
WARNING 03-09 09:18:01 multiproc_worker_utils.py:300] Reducing Torch parallelism from 40 threads to 1 to avoid unnecessary CPU contention. Set OMP_NUM_THREADS in the external environment to tune this value as needed.
INFO 03-09 09:18:01 custom_cache_manager.py:19] Setting Triton cache manager to: vllm.triton_utils.custom_cache_manager:CustomCacheManager
INFO 03-09 09:18:03 cuda.py:178] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 03-09 09:18:03 cuda.py:226] Using XFormers backend.
INFO 03-09 09:18:09 __init__.py:207] Automatically detected platform cuda.
(VllmWorkerProcess pid=8488) INFO 03-09 09:18:09 multiproc_worker_utils.py:229] Worker ready; awaiting tasks
(VllmWorkerProce

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


INFO 03-09 09:18:24 model_runner.py:1115] Loading model weights took 6.7545 GB
(VllmWorkerProcess pid=8488) INFO 03-09 09:18:24 model_runner.py:1115] Loading model weights took 6.7545 GB
(VllmWorkerProcess pid=8488) INFO 03-09 09:18:32 worker.py:267] Memory profiling takes 7.60 seconds
(VllmWorkerProcess pid=8488) INFO 03-09 09:18:32 worker.py:267] the current vLLM instance can use total_gpu_memory (31.75GiB) x gpu_memory_utilization (0.90) = 28.57GiB
(VllmWorkerProcess pid=8488) INFO 03-09 09:18:32 worker.py:267] model weights take 6.75GiB; non_torch_memory takes 0.32GiB; PyTorch activation peak memory takes 2.07GiB; the rest of the memory reserved for KV Cache is 19.43GiB.
INFO 03-09 09:18:32 worker.py:267] Memory profiling takes 7.61 seconds
INFO 03-09 09:18:32 worker.py:267] the current vLLM instance can use total_gpu_memory (31.75GiB) x gpu_memory_utilization (0.90) = 28.57GiB
INFO 03-09 09:18:32 worker.py:267] model weights take 6.75GiB; non_torch_memory takes 0.29GiB; PyTorch ac

Capturing CUDA graph shapes:  80%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                                    | 28/35 [00:22<00:05,  1.28it/s]

(VllmWorkerProcess pid=8488) INFO 03-09 09:18:58 custom_all_reduce.py:226] Registering 2275 cuda graph addresses


Capturing CUDA graph shapes: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35/35 [00:27<00:00,  1.25it/s]

INFO 03-09 09:19:03 custom_all_reduce.py:226] Registering 2275 cuda graph addresses
INFO 03-09 09:19:03 model_runner.py:1562] Graph capturing finished in 28 secs, took 1.06 GiB
(VllmWorkerProcess pid=8488) INFO 03-09 09:19:03 model_runner.py:1562] Graph capturing finished in 28 secs, took 1.06 GiB
INFO 03-09 09:19:03 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 39.56 seconds


In [17]:
def generate_response_vllm(query,pdf_path):
    """
    Generates a response using vLLM based on retrieved chunks.

    Args:
        query (str): The user’s question.
        retrieved_chunks (dict): The retrieved chunks from ChromaDB.

    Returns:
        str: The LLM-generated response.
    """
    # Combine retrieved chunks as context
    retrieved_chunks = your_best_answer(pdf_path,query)
    
    context = "\n\n".join(retrieved_chunks["documents"][0])

    # Construct the final prompt
    prompt = f"""
    You are an AI assistant answering questions based on the given context.
    Format your output in a readable manner. Do not generate output that is not part of the context.
    Generate brief output is there are no restrictions specified in the question.

    Context:
    {context}

    Question:
    {query}

    Answer:
    """

    # Define sampling parameters (adjust for better control)
    sampling_params = SamplingParams(max_tokens=512, temperature=0.1, top_p=0.9)

    # Generate response using vLLM
    outputs = qa_model.generate([prompt], sampling_params)

    return outputs[0].outputs[0].text  # Extract the generated text

In [18]:
pdf_path = './ias_test_file.pdf'
query = 'What is discussed about article 110?'

In [20]:
response = generate_response_vllm(query,pdf_path)
print("---- LLM Response: ----\n", response)

Insert of existing embedding ID: chunk_0
Add of existing embedding ID: chunk_0


Generated 38 embeddings with shape torch.Size([38, 384])


Insert of existing embedding ID: chunk_1
Add of existing embedding ID: chunk_1
Insert of existing embedding ID: chunk_2
Add of existing embedding ID: chunk_2
Insert of existing embedding ID: chunk_3
Add of existing embedding ID: chunk_3
Insert of existing embedding ID: chunk_4
Add of existing embedding ID: chunk_4
Insert of existing embedding ID: chunk_5
Add of existing embedding ID: chunk_5
Insert of existing embedding ID: chunk_6
Add of existing embedding ID: chunk_6
Insert of existing embedding ID: chunk_7
Add of existing embedding ID: chunk_7
Insert of existing embedding ID: chunk_8
Add of existing embedding ID: chunk_8
Insert of existing embedding ID: chunk_9
Add of existing embedding ID: chunk_9
Insert of existing embedding ID: chunk_10
Add of existing embedding ID: chunk_10
Insert of existing embedding ID: chunk_11
Add of existing embedding ID: chunk_11
Insert of existing embedding ID: chunk_12
Add of existing embedding ID: chunk_12
Insert of existing embedding ID: chunk_13
Add 

✅ Stored embeddings in ChromaDB!


Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.23s/it, est. speed input: 204.55 toks/s, output: 53.05 toks/s]

---- LLM Response: ----
  Article 110 of the Indian Constitution outlines the criteria for a bill to be classified as a Money Bill. Discussions revolve around concerns of misuse of this provision, potential violations, the role of the Speaker in certification, and the need for greater transparency and accountability. Regular reviews and public awareness are also suggested to address these issues. Key opinions include concerns raised by the Supreme Court in the Aadhaar case regarding the potential for the government to bypass the Rajya Sabha and undermine Parliament's bicameral nature.
